In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
dataframe = pd.read_csv("train.csv")
dataframe.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [3]:
dataframe['Gender'].replace(['Male','Female'],[0,1], inplace=True)

In [4]:
dataframe['Geography'].replace(['France', 'Spain', 'Germany'],[0,1,2], inplace=True)

In [5]:
train_df = dataframe.copy()
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,0,0,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,0,0,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,0,0,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,0,0,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,1,0,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [6]:
train_features = train_df[["CreditScore","Geography","Gender","Age","Tenure","Balance","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary","Exited"]]
train_features

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,0,0,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,0,0,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,0,0,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,0,0,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,1,0,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...
165029,667,1,1,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,792,0,0,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,565,0,0,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,554,1,1,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [7]:
train, test = train_test_split(train_features, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

105621 train examples
26406 validation examples
33007 test examples


In [8]:
def df_to_dataset(train_features, shuffle=True, batch_size=32):
  train_features = train_features.copy()
  labels = train_features.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(train_features), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [9]:
batch_size = 500 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [10]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [11]:
example_batch = next(iter(train_ds))[0]

In [12]:
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [13]:
feature_columns = []

for header in ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']:
    feature_columns.append(tf.feature_column.numeric_column(header))
    
feature_columns

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='HasCrCard', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='IsActiveMember', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [14]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [15]:
batch_size = 105621 
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100


1/1 [==============================] - 2s 2s/step - loss: 7292.1416 - accuracy: 0.2667 - val_loss: 2317.9624 - val_accuracy: 0.5921
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 2597.2881 - accuracy: 0.5918 - val_loss: 1589.4373 - val_accuracy: 0.7849
Epoch 3/100
1/1 [==============================] - 1s 993ms/step - loss: 1824.1511 - accuracy: 0.7526 - val_loss: 2797.2085 - val_accuracy: 0.7849
Epoch 4/100
1/1 [==============================] - 1s 1000ms/step - loss: 2736.3687 - accuracy: 0.7858 - val_loss: 3393.4377 - val_accuracy: 0.7849
Epoch 5/100
1/1 [==============================] - 1s 981ms/step - loss: 3309.5681 - accuracy: 0.7889 - val_loss: 3559.2546 - val_accuracy: 0.7849
Epoch 6/100
1/1 [==============================] - 1s 981ms/step - loss: 3475.6428 - accuracy: 0.7890 - val_loss: 3421.4792 - val_accuracy: 0.7849
Epoch 7/100
1/1 [==============================] - 1s 994ms/step - loss: 3336.9375 - accuracy: 0.7890 - val_loss: 3065.5256 - val_accur

In [18]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 160ms/step - loss: 212.6274 - accuracy: 0.7877
Accuracy 0.787742018699646
